In [ ]:
#Importamos las bibliotecas 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import timeit

In [ ]:
#Cargamos el archivo a utilizar 
data = pd.read_csv("data/events.csv", low_memory=False)

In [ ]:
data.info()

Pasamos las fechas a formate Datetime para poder trabajar mejor con estas

In [ ]:
import calendar

data['timestamp'] = pd.to_datetime(data['timestamp'])
data['anio'] = data['timestamp'].dt.year
data['mes'] = data['timestamp'].dt.month
data['mes_nombre'] = data['mes'].apply(lambda x: calendar.month_abbr[x])
data['dia'] = data['timestamp'].dt.day
data['hora'] = data['timestamp'].dt.hour
data['date'] = data['timestamp'].dt.date
data['dia del anio'] = data['timestamp'].dt.dayofyear

In [ ]:
data.head(5)

Dado que en el set de datos hay informacion del lugar geografico desde el cual se originan los eventos se  analiza esta informacion.

In [ ]:
#Se ve cuantas ciudades distintas hay en los datos
len(data.city.dropna().unique())

In [ ]:
#Se calcula la cantidad de NaNs para determinar si los datos tienen sentido
cities = data.city.dropna().unique()
cant_nans = len(data) - data['city'].count()
print("cantidad nans: " + str(cant_nans))
print("porcentaje nans: " + str(cant_nans/len(data)))

A pesar de que haya mucha cantidad de Nans en los datos, esto no indica datos erroneos. Los NaN se deben a que no todos los eventos guardan la misma informacion. La informacion de la ciudad y pais solo se ve para el evento de 'visited site', y por esto el resto de los eventos no la contienen. Como se ve a continuacion, si se filtra por el evento en cuestion se ve que no hay valores nulos para las columnas 'city', 'region' y 'country'.

In [ ]:
data[ data['event'] == 'visited site' ].info()

Se busca en que paises se genera mas actividad.

In [ ]:
data.country.value_counts(ascending=True).plot(kind='barh', figsize=(16,10))

Como se ve en el grafico Brasil es el pais desde el cual se producen mas ingresos al sitio. Por lo cual decimos analizar los datos segun su horario. 

In [ ]:
data.country.value_counts(normalize = True).head()

Como se puede ver el 96% de las entradas son desde Brasil, se considera que el timestamp está en zona horaria BRT/ART, por lo cual se manejan los horarios como están, sin verificar la zona horaria del evento . Los valores que presenten un dato erroneo en este caso son despreciables dado que representan un porcentaje bajo de los datos.

Hay una gran cantidad de datos con Country 'unknown', seria interesante analizar si estos se corresponden con city 'unknown' o si hay algun valor del cual se pueda inferir el pais de origen del evento.

In [ ]:
data.city.value_counts(normalize=True).head()

In [ ]:
country_with_city_unknown = data.loc[data.city == 'Unknown']['country']
print("Cantidad de citys unknown: " + str(country_with_city_unknown.size))
print(country_with_city_unknown.unique())

In [ ]:
city_with_country_unknown = data.loc[data.country == 'Unknown']['city']
print("Cantidad de countrys unknown: " + str(city_with_country_unknown.size))
print(city_with_country_unknown.unique())

Se puede ver que los paises 'unknown' se corresponden con city 'unknown'.

In [ ]:
del(country_with_city_unknown)
del(city_with_country_unknown)

Mapas
Se buscan las coordenadas de las ciudades para poder graficarlas en un mapa y ver visualmente dónde están los clientes. Se podria dibujar círculos con radio en funcion de la cantidad de compras del lugar, para tener una idea de la importancia de cada lugar para el estudio de mercado.

El df que se lee a continuacion tiene las coordenadas de las ciudades que aparecen en el set de datos. Leer directamente el nuevo dataset con la info guardada y seguir a partir de ahi. Para ver el procesamiento detalladamente, ver notebook correspondiente.

In [ ]:
cities_with_coordinates = pd.read_csv('/home/miki_mustard/Datosnot/7506_OrganizacionDeDatos_TP1/data/cities_w_coords_with_completed_nans.csv', index_col=False)

In [ ]:
# create cities df
cities_df = pd.DataFrame(cities, columns=['city'])

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="oiudsf")

In [ ]:
from geopy.exc import GeocoderTimedOut
import time

def get_coords(index, x):
    if index % 50 == 0:
        time.sleep(60)
    res = geolocator.geocode(x)
    try:
        return [res.latitude, res.longitude]
    except GeocoderTimedOut:
        return get_coords(x)
    except:
        return [None, None]

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
cities_df.shape

In [ ]:
cities_with_coordinates = pd.read_csv('/home/miki_mustard/Datosnot/7506_OrganizacionDeDatos_TP1/data/cities_w_coords_with_completed_nans.csv', index_col=False)

In [ ]:
import folium

In [ ]:
from folium.plugins import MarkerCluster

avg_lat = cities_with_coordinates['city_lat'].mean()
avg_long = cities_with_coordinates['city_long'].mean()

m = folium.Map(location=[0, 0], zoom_start=2)

marker_cluster = MarkerCluster().add_to(m)

for lat, lon, city in zip(cities_with_coordinates.city_lat, cities_with_coordinates.city_long, cities_with_coordinates.city):
    folium.Marker(location=[lat, lon]).add_to(marker_cluster)

m

Como se dijo anteriormente hay informacion que se muestra solo en determinados eventos y no se repite en otros ingresos del mismo usuario. A continuacion se analiza este tema para poder determinar el estado de los datos.

In [ ]:
data.info()

Con lo que obtenido mediante el uso de info() podemos ver que hay algunas columnas que no tienen ningún NAN, estas son:

timestamp
event
person
Tiene sentido que ninguna tenga NAN, ya que en todos los casos corresponde tener esa información.

Ahora, veamos las columnas que tienen alguna densidad de NAN. En principio puede haber NANs porque esa información no corresponde, podemos pensar que el tipo de evento es el que define qué valores corresponden y cuales no. Recordemos los tipos de eventos que teníamos:

In [ ]:
data['event'].value_counts()

Por ejemplo, vemos que tenemos 50957 eventos que corresponden al search engine hit y viendo devuelta lo que obtuvimos con info() también podemos apreciar que hay en la columna de searchengine la misma cantidad de no NANs. Esto se corresponde con lo que habíamos pensado de que a determinados eventos le corresponden determinada información, y por eso en este caso en todas las demás entradas habría NANs, ya que realmente no corresponde.

Estudiemos más a fondo esto. Primero veamos correspondencias entre event y la información en ciertas columnas.

Correspondencia event-informacion

In [ ]:
# Saco las columnas que no me interesan.
filtered = data.drop(['timestamp','person'], axis='columns')

In [ ]:
def columnas_sin_nans_para_event(dataframe, event):
    return dataframe.loc[dataframe.event == event].iloc[0].dropna().index

In [ ]:
# Vemos las columnas que tienen información (ningún NAN) para cada tipo de evento

for event in data.event.unique():
    print(event + ': ' + str(list(columnas_sin_nans_para_event(filtered, event).drop('event'))) + '\n')

Se puede ver como para cada evento tenemos cierta información que tiene sentido tener y cierta que no. De ahí varios NANs que nos aparecen en algunas columnas.

Pero ahora veamos si todos los NANs en las columnas son porque no corresponde tener esa info para el evento o si hay alguna clase de "fuga" de NANs donde sí corresponde tener la información.

In [ ]:
del(filtered)

# Analizamos la relacion general entre los eventos posibles. 

In [ ]:
cantidad_eventos = data['event'].value_counts(ascending=True)

In [ ]:
cantidad_eventos.plot(kind='barh')

Se puede ver como el evento viewed product es el mas frecuente, siendo este el evento que representa cuando un usuario ingresa a una pagina de producto. 
Tiene sentido que del evento Lead haya muy poca cantidad dado que representa cuando un usuario ingresa su mail para ser notificado en caso de renovar stock de cierto producto.  
Un evento de particular interes es el de Conversion, dado que indica que se efectuo una compra. En el grafico se puede ver que hay muy pocas conversiones. El evento checkout tambien es importante dado que indica que el usuario selecciono un dispositivo para comprar. 
Se puede graficar la evolucion de los eventos a traves del tiempo.

In [ ]:
data.groupby('mes').count()['person'].plot(kind='barh')

Hay significativamente mas datos del mes 5. Si los datos son representativos, se podria concluir que hubo mucho mas trafico durante el quinto mes (casi el doble que durante el sexto, que ademas tiene aproximadamente el doble que el cuarto, que es el que mas tiene entre los primeros cuatro.
Surge asi el interrogante: que es lo que hizo que este mes haya tenido mas trafico?


In [ ]:
# Se obtiene el día de la semana, se cuenta y se vuelca en un gráfico
days = {6: "Domingo",0: "Lunes",1:"Martes",2: "Miércoles",3: "Jueves",4: "Viernes",5: "Sábado"}

# Se guardan dias de la semana en el orden deseado
sorter = ["Domingo", "Sábado", "Viernes", "Jueves", "Miércoles", "Martes", "Lunes"]
sorterIndex = dict(zip(sorter,range(len(sorter))))

# se crea columna con info del dia de la semana respetando el nombre
data["dia_semana"] = data["timestamp"].dt.dayofweek
data["dia_semana"] = data["dia_semana"].apply(lambda x: days[x])

In [ ]:
# Se crea df con cantidad de eventos agrupados por dia de la semana
dfEventosAgrupado = data[['event','dia_semana']].groupby("dia_semana").count()
dfEventosAgrupado["dia"] = dfEventosAgrupado.index

# ordenamiento coherente para evitar el default (alfabetico)
dfEventosAgrupado["dia"] = dfEventosAgrupado["dia"].map(sorterIndex)

grafico = dfEventosAgrupado.sort_values("dia").plot(kind = "barh", figsize=(12,5))
grafico.legend_.remove()
grafico.set_title("Cantidad de eventos por día de la semana", fontsize=20)
grafico.set_xlabel("Día de la semana", fontsize=15)
grafico.set_ylabel("Cantidad de eventos", fontsize=15)

Se puede observar que hay menos trafico los fines de semana. Fuera de eso, los niveles se mantienen similares. Se puede suponer que esto se debe a que los fines de semana los usuarios ocupan mas tiempo en actividades sociales y que durante la semana navegan en este tipo de sitios despues del trabajo (o durante? Seguro que no!).

In [ ]:
# Se obtiene la hora, se cuenta y se vuelca en un gráfico
grafico = data["timestamp"].dt.hour.value_counts().sort_index().plot(figsize=(10,7))
grafico.set_title("Cantidad de eventos por hora del día", fontsize=20)
grafico.set_xlabel("Hora del día", fontsize = 15)
grafico.set_ylabel("Cantidad de eventos", fontsize = 15)

Como era de esperar, durante la noche se registra una baja en la actividad del sitio.

In [ ]:
eventos_en_t  =  pd.DataFrame()
eventos_en_t['date'] = data['timestamp'].dt.date
eventos_en_t['checkout'] = data['event']
eventos_en_t['conversion'] = eventos_en_t['checkout'].str.contains('conversion')
eventos_en_t['ad campaign hit'] = eventos_en_t['checkout'].str.contains('ad campaign hit')
eventos_en_t['viewed product'] = eventos_en_t['checkout'].str.contains('viewed product')
eventos_en_t['visited site'] = eventos_en_t['checkout'].str.contains('visited site')

eventos_en_t['checkout'] = eventos_en_t['checkout'].str.contains('checkout')
grouped = eventos_en_t.groupby('date').sum()

In [ ]:
ax = grouped.plot(figsize=(10,10))

Se puede ver como el evento mas predominate es el de 'viewed product'. Analizamos el resto de los eventos sin tenerlo en cuenta.

In [ ]:
ax = grouped.plot(figsize=(10,10))

Los eventos 'visited site' y 'ad campaign hit' se distribuyen a lo largo del tiempo de forma muy similar.  Mientras que 'checkout' es menos frecuente, y aun menos 'conversion'.

In [ ]:
#Si analizamos los mismos datos con un PairGrid
from  pandas.plotting  import scatter_matrix
#En general por fecha
scatter_matrix(grouped, alpha=0.2, figsize=(15, 15), diagonal='kde')

Se obtiene la misma relacion "lineal" entre los eventos 'ad campaign hit' y 'visited site', mientras que entre 'visited site' y 'checkouts' es mas frecuente el primero. Es de especial importancia la relacion entre 'choeckout' y 'conversion'. En este grafico se puede observar como los 'chekout' son mas frecuentes. esto se puede deber a que un usuario busca certo dispositivo y luego no efectua la compra, asi generando uno o mas eventos de 'checkout' y no un evento 'conversion'.

## Analisis del evento 'conversion'

In [ ]:
eventos_conversion = data.loc[data.event == 'conversion']
eventos_conversion.shape

Cantidad de conversiones durante el año

In [ ]:
conversion_per_date = eventos_conversion.timestamp.dt.date.value_counts()

In [ ]:
#g = plt.hist(x = conversion_per_date.index, weights = conversion_per_date.values)
#me tira error

Cantidad de conversiones por mes

In [ ]:
conversion_per_month = eventos_conversion.mes.value_counts()
conversion_per_month

In [ ]:
g  =  sns.barplot(x=conversion_per_month.values, y=conversion_per_month.index, orient='h')
g.set_title("Conversiones por mes", fontsize=20)
g.set_xlabel("Numero de conversiones", fontsize=15)
g.set_ylabel("Mes", fontsize=15)

Con lo anterior vemos que durante los meses 3-5 son los que se tuvo más eventos de conversion.

In [ ]:
del(conversion_per_month)

Cantidad de conversiones según día de la semana y hora

In [ ]:
table = eventos_conversion.pivot_table(index = 'hora', columns = 'dia_semana', values = 'dia', aggfunc = 'count')
table

In [ ]:
g = sns.heatmap(table,  cmap="YlGnBu")
g.set_title("Eventos de conversion (compra) por día de la semana y hora", fontsize=22)
g.set_xlabel("Día de la semana",fontsize=18)
g.set_ylabel("Hora del día", fontsize=18)

Esta informacion se corresponde con la obtenida anteriormente para los eventos en general. Mostrando nuevamente que dentro de las horas de la mañana 3-11 hay casi nada de actividad y los días de fin de semana hay menos actividad.

In [ ]:
del(table)
del(g)

## Conversiones segun el modelo

In [ ]:
# Vemos como se distribuyen los NaN
print("Cantidad de nans para eventos de compra: " + str(eventos_conversion.model.size - eventos_conversion.model.count()))

No hay ningun NaN en los eventos de conversion para el modelo. Esto tiene sentido dado que el evento de conversion se genera al comprar un producto determinado, por lo cual se tiene que generar la informacion correspondiente a dicho dispositivo. 

Veamos los 10 modelos más comúnes en el evento conversion. Hay demasiados modelos como para enfocarnos en ver todos los modelos y las distribuciones de cantidad de veces que aparecen en el evento.

In [ ]:
conversion_per_model = eventos_conversion.model.str.lower().value_counts().head(10)
conversion_per_model

In [ ]:
eventos_conversion.model.str.lower().value_counts(normalize = True).head(10)

In [ ]:
g = sns.barplot(x=conversion_per_model.values, y=conversion_per_model.index, orient='h')
g.set_title("Conversiones por modelo (10 mas comprados)", fontsize=15) # o mas vendidos?
g.set_xlabel("Numero de conversiones", fontsize=12)
g.set_ylabel("Modelo", fontsize=12) #o modelos??

Se ve claramente como los primeros 3 modelos tienen un valor de conversiones considerablemente mayor al resto de los modelos. El Samsung Galaxy J5 representa mas de doble de los ultimos 7 modelos. 

In [ ]:
del(conversion_per_model)
del(g)

# Analisis por caracteristicas de celulares

## Storage

In [ ]:
eventos_check_out  = data.loc[data.event == 'checkout']
eventos_check_out.head()

In [ ]:
check_out_per_storage = eventos_check_out.storage.value_counts()
check_out_per_storage

In [ ]:
mes = {1:"Jan",2: "Feb",3: "Mar",4: "May",5: "Jun"}

# guardo meses en el orden deseado
sorter = ["Jan", "Feb","Mar", "Apr", "May", "Jun"]
sorterIndex = dict(zip(sorter,range(len(sorter))))

# creo df con cantidad de eventos agrupados por dia de la semana
#dfEventosAgrupado = df[['event','dia_semana']].groupby("dia_semana").count()
#dfEventosAgrupado["day"] = dfEventosAgrupado.index

# ordenamiento coherente para evitar el default (alfabetico)
#dfEventosAgrupado["day"] = dfEventosAgrupado["day"].map(sorterIndex)

#grafico = dfEventosAgrupado.sort_values("day").plot(kind = "barh")
#grafico.legend_.remove()
#grafico.set_title("Cantidad de eventos por día de la semana", fontsize=18)
#grafico.set_xlabel("Día de la semana", fontsize=12)
#grafico.set_ylabel("Cantidad de eventos", fontsize=12)

In [ ]:
viewed_product_distribucion = data[ data['event'] == 'viewed product'].groupby(['dia', 'mes'])['mes_nombre']
vpd = viewed_product_distribucion.agg('count').unstack()
vpd.plot()

In [ ]:
vp = data[ data['event'] == 'viewed product']

In [ ]:
for_heatmap = vp.pivot_table(index='mes', columns='dia', values='timestamp', aggfunc='count')
for_heatmap

In [ ]:
plt.figure(figsize=(18, 5))
g = sns.heatmap(for_heatmap,  cmap="YlGnBu",square=True,  linewidths=.5)
g.set_title("Cantidad de viewed product por dia segun el mes", fontsize=22)
g.set_xlabel("Dia del evento",fontsize=18)
g.set_ylabel("Mes del evento", fontsize=18)

Es importante comparar los eventos checkout y conversion

In [ ]:
#Primero filtro los eventos que me interesan
e = data[ (data['event'] != 'viewed product') & (data['event'] != 'lead')]
#Group By mes y evento
groupbymesyevento = e.groupby(['mes', 'event'])

#Without the hierarchical indexing
print ("[Unstacking]" )
regimentbucketclicksum = groupbymesyevento['person'].agg('count').unstack()

regimentbucketclicksum.plot(kind = 'bar', title = 'Frecuencia de los eventos por mes', figsize=(10,10))

plt.ylabel('cantidad')
plt.show()

In [ ]:
p = pd.DataFrame(data[ ['person', 'event']])
grouped = p.groupby('person')['event']


In [ ]:
p['ad campaign hit'] = p['event'].str.contains('ad campaign hit')
p

In [ ]:
#data['event'] = data['event'].astype('category')

In [ ]:
data.columns

In [ ]:
ap = pd.crosstab(data['person'], data['event'])

In [ ]:
ap.head()

In [ ]:
ap.describe()

In [ ]:
print( 'promedio: ', ap.mean() , ",desviacion estandar: " , ap.std())

In [ ]:
ap.agg(['mean', 'std'])

In [ ]:
ap.mean().plot(kind='barh')

In [ ]:
# Nos interesa ver si un usuario realizo o no un evento, no la cantidad de veces que los realiza.
# Para ver esto los pasamos a formato de booleanos.
ap = ap.astype('bool')
ap.head()

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori_result = apriori(ap, min_support=0.0, use_colnames=True, max_len=2)

In [ ]:
apriori_result.head()

In [ ]:
# Guardamos soportes para cada tipo de evento
support_per_event = apriori_result[apriori_result['itemsets'].apply(lambda x: len(x) == 1)]

support_per_event = support_per_event.rename(columns={'itemsets':'event'})
support_per_event.head()

In [ ]:
support_per_event['event'] = support_per_event['event'].apply(lambda x: x[0])

In [ ]:
support_per_event.head()

In [ ]:
support_per_event = support_per_event.set_index('event').T

In [ ]:
support_per_event.head()

In [ ]:
apriori_result = apriori_result[apriori_result['itemsets'].apply(lambda x: len(x) == 2)]

In [ ]:
apriori_result.head(50)

In [ ]:
apriori_result = apriori_result.sort_values('support',ascending=False)
apriori_result.head()

In [ ]:
apriori_result = apriori_result.reset_index(drop=True)
apriori_result.head()

In [ ]:
apriori_result.count()

In [ ]:
# Borramos rows con soporte = 0 (es decir, duplas que no se hayan dado en nuestros datos)
apriori_result = apriori_result[apriori_result['support'] > 0.0]

In [ ]:
apriori_result.count()

In [ ]:
# Vamos a analizar las 20 duplas con mayor soporte (frecuencia de aparicion en los datos analizados)
apriori_result.nlargest(20,'support')

In [ ]:
apriori_result[ apriori_result['itemsets'] == {'checkout', 'conversion'} ]

In [ ]:
apriori_result.plot()

In [ ]:
working_data = apriori_result.rename(columns={'itemsets':'event1'})
working_data.head()

In [ ]:
df2 = pd.DataFrame(list(working_data['event1']), columns = ['event1', 'event2'])
df2.head()

In [ ]:
df2['support'] = working_data['support']
df2.head()

In [ ]:
df1 = pd.DataFrame(list(working_data['event1']), columns = ['event2', 'event1'])
df1.head()

In [ ]:
df1['support'] = working_data['support']
df1.head()

In [ ]:
df1.info()

In [ ]:
# Uno los 2 DF:
apriori_result = pd.concat([df1, df2]).reset_index(drop=True)
apriori_result.head()

In [ ]:
apriori_result.info()

In [ ]:
support_per_event

In [ ]:
#Calculo de la confianza (la calculamos RESPECTO AL EVENTO EVENT1). Esto es support de la dupla dividido support individual de una de las 2 categorias de 
# dicha dupla. Es decir, de los usuarios que realizaron el evento individual, cuantos realizaron tambien el otro evento de la dupla. 
#(vamos a buscar el support individual al DF que teniamos dedicado para eso -support_per_event-)
for index, row in apriori_result.iterrows():
    apriori_result.at[index, 'confidence'] = apriori_result.at[index, 'support'] / support_per_event.at['support', frozenset({row['event1']})]

In [ ]:
apriori_result.head()

In [ ]:
# Ordenamos para ver cuales son los eventos más "conectados/relacionados" (duplas de mayor confianza)
apriori_result = apriori_result.sort_values('confidence', ascending=False)
apriori_result.head()

In [ ]:
# Hay varios con una confianza del 100%, esto se debe a que el soporte del la dupla y el del individual 
# son iguales, y esto se da para cada tipo de evento con el que se relaciona el checkout (por como se manejaron
# los datos, no son casos verídicos). Vamos a sacar estos casos.
apriori_result = apriori_result[apriori_result['confidence'] < 1.0]
apriori_result.head()

In [ ]:
# Vamos a analizar las 20 duplas con la mayor confianza (de la dupla respecto a category1)
apriori_result.nlargest(20, 'confidence')

In [ ]:
apriori_result['tuple'] = apriori_result['event1'] + ' -> ' + apriori_result['event2']

In [ ]:
apriori_result.head()

In [ ]:
working_data = apriori_result[['tuple','confidence']].nlargest(20,'confidence')

fig, ax = plt.subplots(figsize=(20,15))

ax = sns.barplot(x=working_data['confidence'], y=working_data['tuple'], orient='h', edgecolor="black", linewidth=0.5)
ax.set_title('Cercanía de eventos: usuarios de 1er evento, que % realizó también el 2do evento', fontsize=25)
ax.set_xlabel('%', labelpad=15, size=20)
ax.set_ylabel('Eventos (1ro -> 2do)', labelpad=25, size=20)
ax.set_yticklabels(ax.get_yticklabels(), size=13)
ax.grid(axis='x')
ax.set_xbound(0.7, 1.0)
ax.set_xticklabels(['65','70','75','80','85','90','95','100'], size=13)

ax.title.set_position([0.3, 1.05])

In [ ]:
# Ahora analicemos por truplas
apriori_result = apriori(ap, min_support=0.0, use_colnames=True, max_len=3)

In [ ]:
apriori_result = apriori_result[apriori_result['itemsets'].apply(lambda x: len(x) == 3)]
apriori_result.head()

In [ ]:
apriori_result = apriori_result.sort_values('support',ascending=False)
apriori_result.head()

In [ ]:
apriori_result = apriori_result.reset_index(drop=True)
apriori_result.head()

In [ ]:
# Hacemos un dataframe con columnas por los eventos, para facilidad del manejo de los datos
df = pd.DataFrame(list(apriori_result['itemsets']), columns = ['event1', 'event2', 'event3'])
df.head(20)

In [ ]:
# Le agregamos la columna con el support
df['support'] = apriori_result['support']
df

In [ ]:
df = df.sort_values('support', ascending=False)
df.head(10)

In [ ]:
# Vamos a estudiar las truplas que contienen el evento conversion, ya que nos interesa
# ver el movimiento de los usuarios que llegaron a comprar algo.
df = df.loc[(df.event2 == 'conversion') | (df.event1 == 'conversion') | (df.event3 == 'conversion')]
df

In [ ]:
# Como a cada conversion siempre va a estar asociado un checkout, descartamos las truplas
# con este evento.
df = df.loc[(df.event2 != 'checkout') & (df.event1 != 'checkout') & (df.event3 != 'checkout')]
df.head()

In [ ]:
df = df.sort_values('support', ascending=False)
df.head(10)

In [ ]:
df['tuple'] = df['event1'] + ',' + df['event2']+ ',' + df['event3']

In [ ]:
working_data = df.nlargest(20,'support')

fig, ax = plt.subplots(figsize=(15,20))

ax = sns.barplot(x=working_data['support'], y=working_data['tuple'], orient='h', edgecolor="black", linewidth=0.5)
ax.set_title('Truplas de eventos conteniendo conversion (ventas) con mayor soporte', fontsize=25)
ax.set_xlabel('Soporte', labelpad=15, size=20)
ax.set_ylabel('Truplas', rotation=0, labelpad=15, size=20)
ax.set_yticklabels(ax.get_yticklabels(), size=25)
ax.set_xticklabels(['0', '5', '10', '15', '20', '25'], size=20)
ax.grid(axis='x')

ax.title.set_position([0.35, 1.05])

In [ ]:
ap.plot()

In [ ]:
# Saco las columnas que no me interesan.
filtered = data.drop(['timestamp','person'], axis='columns')

In [ ]:
def columnas_sin_nans_para_event(dataframe, event):
    return dataframe.loc[dataframe.event == event].iloc[0].dropna().index

In [ ]:
for event in data.event.unique():
    print(event + ': ' + str(list(columnas_sin_nans_para_event(filtered, event).drop('event'))))

In [ ]:
ind = data.columns.drop(['timestamp', 'person','event'])
ind

In [ ]:
d = {'ad campaign hit' : pd.Series([1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0], index=ind),
     'visited site' : pd.Series([0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1], index= ind),
     'viewed product' : pd.Series([0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], index= ind),
     'checkout' : pd.Series([0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], index= ind),
     'generic listing' : pd.Series([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], index= ind),
      'searched engine hit' : pd.Series([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0], index= ind),
      'brand listing' : pd.Series([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], index= ind),
      'searched products' : pd.Series([0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0], index= ind),
      'conversion' : pd.Series([0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], index= ind),
      'staticpage' : pd.Series([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0], index= ind),
      'lead' : pd.Series([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], index= ind)}

In [ ]:
df = pd.DataFrame(d)
df


In [ ]:
plt.figure(figsize=(18, 5))
g = sns.heatmap(df.T,linewidths=0.5, linecolor='white', square=True,cmap="Greens",cbar=False)
g.set_title("Columnas con datos no nulos segun evento", fontsize=22)
g.set_xlabel("Columnas con datos no nulos",fontsize=18)
g.set_ylabel("Eventos", fontsize=18)

In [ ]:
usuarios_nuevos = data[ data['new_vs_returning'] == 'New']

In [ ]:
data['new_vs_returning'].value_counts()

In [ ]:
a = data[['person', 'timestamp','city','event','new_vs_returning']].dropna()
a